<a href="https://colab.research.google.com/github/Alfikriangelo/DeepLearningTasks/blob/main/2ndWeekTask/SecondHand_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PyTorch**

In [6]:
#import library
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# 1. MEMUAT DATASET
data = pd.read_csv('sample_data/secondhanddataset.csv')

In [8]:
# 2. PRA-PROSES DATA
# Pisahkan fitur (X) dan target harga (y)
X = data.drop(columns=['current price']).values
y = data['current price'].values.reshape(-1, 1)  # Bentuk y harus (n_samples, 1)

In [9]:
# Membagi data menjadi data latih (80%) dan data uji (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Normalisasi fitur agar memiliki distribusi yang lebih stabil
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

In [11]:
# Normalisasi target agar model lebih cepat konvergen
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [12]:
# Konversi data ke Tensor PyTorch
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [13]:
# 3. MEMBUAT DATASET UNTUK MINI-BATCH TRAINING
batch_size = 32  # Ukuran batch untuk setiap iterasi pelatihan
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [15]:
# 4. MEMBANGUN MODEL MLP
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)  # Lapisan pertama dengan 128 neuron
        self.fc2 = nn.Linear(128, 64)  # Lapisan kedua dengan 64 neuron
        self.fc3 = nn.Linear(64, 1)  # Lapisan output (tanpa aktivasi karena regresi)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Aktivasi ReLU untuk lapisan pertama
        x = torch.relu(self.fc2(x))  # Aktivasi ReLU untuk lapisan kedua
        x = self.fc3(x)  # Output tanpa aktivasi
        return x

In [16]:
# Inisialisasi model dengan jumlah fitur yang sesuai
model = MLP(input_dim=X_train.shape[1])

In [17]:
# 5. MENDEFINISIKAN LOSS FUNCTION DAN OPTIMIZER
criterion = nn.MSELoss()  # Fungsi loss menggunakan Mean Squared Error (MSE)
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Optimizer Adam dengan learning rate kecil agar stabil

In [18]:
# 6. LOOP PELATIHAN MODEL
epochs = 100  # Jumlah iterasi pelatihan
for epoch in range(epochs):
    model.train()  # Mengaktifkan mode pelatihan
    epoch_loss = 0  # Menyimpan akumulasi loss per epoch

    for batch_X, batch_y in train_loader:  # Iterasi melalui batch dalam DataLoader
        optimizer.zero_grad()  # Reset gradien sebelum backpropagation
        outputs = model(batch_X)  # Prediksi dari model
        loss = criterion(outputs, batch_y)  # Hitung loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update bobot model
        epoch_loss += loss.item()  # Simpan total loss

    # Tampilkan loss setiap 10 epoch
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss / len(train_loader):.4f}')

Epoch [10/100], Loss: 0.0057
Epoch [20/100], Loss: 0.0024
Epoch [30/100], Loss: 0.0016
Epoch [40/100], Loss: 0.0011
Epoch [50/100], Loss: 0.0008
Epoch [60/100], Loss: 0.0006
Epoch [70/100], Loss: 0.0005
Epoch [80/100], Loss: 0.0003
Epoch [90/100], Loss: 0.0003
Epoch [100/100], Loss: 0.0002


In [20]:
# 7. EVALUASI MODEL
model.eval()  # Mengaktifkan mode evaluasi
with torch.no_grad():  # Tidak perlu menghitung gradien saat evaluasi
    y_pred = model(X_test)  # Prediksi data uji
    mse = criterion(y_pred, y_test)  # Hitung MSE
    rmse = torch.sqrt(mse)  # Hitung RMSE

    # Konversi hasil prediksi kembali ke skala asli
    y_pred_original = scaler_y.inverse_transform(y_pred.numpy())
    y_test_original = scaler_y.inverse_transform(y_test.numpy())

    # Hitung R-squared dalam skala asli
    y_mean = np.mean(y_test_original)
    ss_total = np.sum((y_test_original - y_mean) ** 2)  # Total Variasi
    ss_res = np.sum((y_test_original - y_pred_original) ** 2)  # Variasi yang tidak dapat dijelaskan
    r_squared = 1 - (ss_res / ss_total)  # R-squared

In [23]:
# 8. MENAMPILKAN HASIL EVALUASI
print(f'MSE: {mse.item():.4f}')
print(f'RMSE: {rmse.item():.4f}')
print(f'R-squared: {r_squared:.4f}')

MSE: 0.0035
RMSE: 0.0590
R-squared: 0.9968


# **Evaluasi Model: MSE, RMSE, dan R-squared**

Setelah melatih model, kita mengevaluasi performanya menggunakan tiga metrik utama:

1. **Mean Squared Error (MSE)**
2. **Root Mean Squared Error (RMSE)**
3. **R-squared (\( R^2 \))**

Hasil evaluasi yang diperoleh:

---

## **1. Mean Squared Error (MSE)**
MSE mengukur rata-rata kesalahan kuadrat antara nilai prediksi dan nilai aktual. Nilai MSE yang lebih kecil menunjukkan bahwa model memiliki error yang lebih rendah.

**Rumus MSE:**
$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

**Interpretasi:**  
- Nilai **MSE = 0.0035** menunjukkan bahwa rata-rata kuadrat selisih antara nilai aktual dan prediksi sangat kecil, yang berarti model cukup akurat.

---

## **2. Root Mean Squared Error (RMSE)**
RMSE adalah akar dari **MSE**, yang memiliki satuan yang sama dengan variabel target. RMSE lebih mudah diinterpretasikan dibandingkan MSE.

**Rumus RMSE:**
$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

**Interpretasi:**  
- Nilai **RMSE = 0.0590** menunjukkan bahwa rata-rata kesalahan prediksi berada dalam kisaran **5.9%** dari skala target setelah dinormalisasi kembali ke skala aslinya.

---

## **3. R-squared (\( R^2 \))**
**R-squared** mengukur seberapa baik model dapat menjelaskan variabilitas dalam data. Nilainya berkisar antara **0 hingga 1**, di mana:
- **\( R^2 = 1 \)** berarti model **sempurna** dalam menjelaskan data.
- **Semakin mendekati 1**, semakin baik model dalam menjelaskan hubungan antara fitur dan target.

**Rumus R-squared:**
$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$
di mana:
$$
\bar{y} = \frac{1}{n} \sum_{i=1}^{n} y_i
$$
adalah rata-rata dari nilai aktual.

**Interpretasi:**  
- Nilai **\( R^2 = 0.9968 \)** berarti model dapat menjelaskan **99.68%** variabilitas dalam data, yang menunjukkan performa yang **sangat baik**.

---

## **Kesimpulan**
Dari hasil evaluasi di atas:
- **MSE dan RMSE bernilai kecil**, menunjukkan bahwa model membuat prediksi dengan error yang sangat rendah.
- **R-squared mendekati 1**, menunjukkan bahwa model sangat baik dalam menjelaskan data.

Secara keseluruhan, model yang telah dibuat memiliki **akurasi yang tinggi dan performa yang sangat baik dalam memprediksi harga barang bekas.** 🚀

# **Tensorflow**

In [24]:
# Import library
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [25]:
# Memuat dataset
data = pd.read_csv('sample_data/secondhanddataset.csv')

In [26]:
# Menangani nilai yang hilang (missing values)
data = data.dropna()

In [27]:
# Memisahkan fitur (X) dan variabel target (y)
X = data.drop(columns=['current price']).values
y = data['current price'].values.reshape(-1, 1)

In [28]:
# Standarisasi fitur (X)
scaler_X = StandardScaler()
X = scaler_X.fit_transform(X)

In [29]:
# Standarisasi variabel target (y)
scaler_y = StandardScaler()
y = scaler_y.fit_transform(y).flatten()

In [30]:
# Membagi data menjadi data latih (train) dan data uji (test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Mendefinisikan model MLP yang lebih baik
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation=tf.keras.layers.LeakyReLU(), input_shape=(X_train.shape[1],)),  # Lapisan pertama dengan 256 neuron
    tf.keras.layers.Dense(128, activation=tf.keras.layers.LeakyReLU()),  # Lapisan kedua dengan 128 neuron
    tf.keras.layers.Dense(64, activation=tf.keras.layers.LeakyReLU()),  # Lapisan ketiga dengan 64 neuron
    tf.keras.layers.Dropout(0.3),  # Menggunakan dropout untuk mengurangi overfitting
    tf.keras.layers.Dense(32, activation=tf.keras.layers.LeakyReLU()),  # Lapisan keempat dengan 32 neuron
    tf.keras.layers.Dense(1)  # Lapisan output dengan 1 neuron (karena ini adalah tugas regresi)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
# Mengompilasi model dengan tingkat pembelajaran (learning rate) yang dioptimalkan
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [33]:
# Menggunakan Early Stopping untuk menghentikan pelatihan jika model tidak mengalami peningkatan
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [34]:
# Melatih model
history = model.fit(X_train, y_train,
                    epochs=200,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping],
                    verbose=1)

Epoch 1/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.4646 - mae: 0.5301 - val_loss: 0.0494 - val_mae: 0.1787
Epoch 2/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0647 - mae: 0.1947 - val_loss: 0.0180 - val_mae: 0.1079
Epoch 3/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.0304 - mae: 0.1393 - val_loss: 0.0100 - val_mae: 0.0809
Epoch 4/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0238 - mae: 0.1175 - val_loss: 0.0126 - val_mae: 0.0916
Epoch 5/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0245 - mae: 0.1185 - val_loss: 0.0065 - val_mae: 0.0655
Epoch 6/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0216 - mae: 0.1117 - val_loss: 0.0082 - val_mae: 0.0715
Epoch 7/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0191 - mae: 0.1034 - val_loss: 0.0046 - val_mae: 0.0538
Epoch 8/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0192 - mae: 0.1066 - val_loss: 0.0070 - val_mae: 0.0681
Epoch 9/200
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss:

In [35]:
# Mengevaluasi model dengan data uji
y_pred = model.predict(X_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [36]:
# Mengonversi prediksi kembali ke skala aslinya
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

In [37]:
# Menghitung metrik evaluasi model
mse = tf.keras.losses.MeanSquaredError()(y_test_original, y_pred_original).numpy()  # Mean Squared Error (MSE)
rmse = np.sqrt(mse)  # Root Mean Squared Error (RMSE)
mae = tf.keras.losses.MeanAbsoluteError()(y_test_original, y_pred_original).numpy()  # Mean Absolute Error (MAE)

In [38]:
# Menghitung koefisien determinasi (R-squared)
y_mean = y_test_original.mean()
ss_total = ((y_test_original - y_mean) ** 2).sum()  # Total sum of squares (variabilitas total)
ss_res = ((y_test_original - y_pred_original) ** 2).sum()  # Residual sum of squares (variabilitas yang tidak dijelaskan)
r_squared = 1 - (ss_res / ss_total)  # Perhitungan R²

In [39]:
# Menampilkan hasil evaluasi model
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')
print(f'MAE: {mae:.4f}')
print(f'R-squared: {r_squared:.4f}')

MSE: 59534604.0000
RMSE: 7715.8672
MAE: 6072.8125
R-squared: 0.9965


# **Evaluasi Model: MSE, RMSE, dan R-squared**

Setelah melatih model, kita mengevaluasi performanya menggunakan tiga metrik utama:

1. **Mean Squared Error (MSE)**
2. **Root Mean Squared Error (RMSE)**
3. **R-squared (\( R^2 \))**

Hasil evaluasi yang diperoleh:

---

## **1. Mean Squared Error (MSE)**
MSE mengukur rata-rata kesalahan kuadrat antara nilai prediksi dan nilai aktual. Nilai MSE yang lebih kecil menunjukkan bahwa model memiliki error yang lebih rendah.

**Rumus MSE:**
$$
MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2
$$

**Interpretasi:**  
- Nilai **MSE = 59,534,604.0000** menunjukkan bahwa rata-rata kuadrat selisih antara nilai aktual dan prediksi cukup besar, yang berarti terdapat beberapa kesalahan dalam prediksi model.

---

## **2. Root Mean Squared Error (RMSE)**
RMSE adalah akar dari **MSE**, yang memiliki satuan yang sama dengan variabel target. RMSE lebih mudah diinterpretasikan dibandingkan MSE.

**Rumus RMSE:**
$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

**Interpretasi:**  
- Nilai **RMSE = 7,715.8672** menunjukkan bahwa rata-rata kesalahan prediksi berada dalam kisaran **7,715 unit dari skala target**, yang bisa menjadi indikasi bahwa model masih memiliki error yang cukup besar dalam skala aslinya.

---

## **3. Mean Absolute Error (MAE)**
MAE mengukur rata-rata selisih absolut antara nilai aktual dan nilai prediksi, memberikan gambaran tentang seberapa besar kesalahan rata-rata model.

**Rumus MAE:**
$$
MAE = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat{y}_i|
$$

**Interpretasi:**  
- Nilai **MAE = 6,072.8125** menunjukkan bahwa secara rata-rata, prediksi model meleset sekitar **6,072 unit** dari nilai sebenarnya.

---

## **4. R-squared (\( R^2 \))**
**R-squared** mengukur seberapa baik model dapat menjelaskan variabilitas dalam data. Nilainya berkisar antara **0 hingga 1**, di mana:
- **\( R^2 = 1 \)** berarti model **sempurna** dalam menjelaskan data.
- **Semakin mendekati 1**, semakin baik model dalam menjelaskan hubungan antara fitur dan target.

**Rumus R-squared:**
$$
R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}_i)^2}{\sum_{i=1}^{n} (y_i - \bar{y})^2}
$$
di mana:
$$
\bar{y} = \frac{1}{n} \sum_{i=1}^{n} y_i
$$
adalah rata-rata dari nilai aktual.

**Interpretasi:**  
- Nilai **\( R^2 = 0.9965 \)** berarti model dapat menjelaskan **99.65%** variabilitas dalam data, yang menunjukkan performa yang **sangat baik** dalam menangkap pola data.

---

## **Kesimpulan**
Dari hasil evaluasi di atas:
- **MSE dan RMSE bernilai cukup besar**, menunjukkan bahwa model masih memiliki error dalam prediksi.
- **MAE juga cukup besar**, mengindikasikan selisih absolut antara nilai aktual dan prediksi masih cukup tinggi.
- **R-squared mendekati 1**, menunjukkan bahwa model sangat baik dalam menjelaskan data meskipun masih ada error yang perlu diperbaiki.

Secara keseluruhan, model memiliki **kemampuan yang sangat baik dalam menjelaskan variabilitas data**, tetapi masih terdapat kesalahan prediksi yang cukup besar, sehingga perlu dilakukan analisis lebih lanjut untuk meningkatkan akurasi model. 🚀
